Zagadnieniem tej pracy domowej było wybranie bazy danych oraz zbudowanie modeli klasyfikacji do niej, tak aby sprawdzić ich działanie. Moja baza danych opisywała wiele różnych
parametrów wina, takich jak kwasowość, cukry, siarczyny oraz alkohol i tym podobne oraz jego ocena. Zadaniem było stworzenie modelu, który na podstawie tych parametrów będzie
w stanie określić, jaką ocena wino otrzymało. W tym celu wykorzystałem 4 różne modele: drzewo decyzyjne, k-Najbliższych sąsiadów, Naive Bayes oraz sieć neuronową. Wszystkie
modele zostały przetestowane na danych treningowych, a następnie na danych testowych. Przetestowane zostało wiele parametrów danych modeli.

Funkcje odpowiadające za wczytanie danych i ich preprocessing:

In [138]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pandas as pd
from sklearn.utils import shuffle

def getData(directory):
    data = pd.read_csv(directory)
    data = shuffle(data)
    data['quality'] = data['quality'].replace([3, 4, 5], 'low')
    data['quality'] = data['quality'].replace([6, 7], 'medium')
    data['quality'] = data['quality'].replace([8, 9], 'high')
    return data

def withDeletion(df):
    label_encoder = LabelEncoder()
    df["type"] = label_encoder.fit_transform(df["type"])
    df = df.dropna()
    return df

def withMean(df):
    label_encoder = LabelEncoder()
    df["type"] = label_encoder.fit_transform(df["type"])
    df = df.fillna(df.mean(numeric_only=True))
    return df

def normalization(df):
    scaler = MinMaxScaler()
    columns_to_normalize = df.columns[:-1]
    df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
    return df

print("Wczytane i pomieszane dane: ")
print(getData("./Data/winequality.csv").head())
print("Dane po normalizacji: ")
print(normalization(withMean(getData("./Data/winequality.csv"))).head())

Wczytane i pomieszane dane: 
       type  fixed acidity  volatile acidity  citric acid  residual sugar  \
5253    red            6.7              0.75         0.01            2.40   
1688  white            6.7              0.25         0.26            1.55   
6259    red            8.3              0.85         0.14            2.50   
4844  white            4.8              0.29         0.23            1.10   
3417  white            6.7              0.64         0.30            1.20   

      chlorides  free sulfur dioxide  total sulfur dioxide  density    pH  \
5253      0.078                 17.0                  32.0  0.99550  3.55   
1688      0.041                118.5                 216.0  0.99490  3.55   
6259      0.093                 13.0                  54.0  0.99724  3.36   
4844      0.044                 38.0                 180.0  0.98924  3.28   
3417      0.030                 18.0                  76.0  0.98920  3.16   

      sulphates  alcohol quality  
5253      

Są tu dwie funkcje odpowiadające za preprocessing danych. Pierwsza z nich usuwa wszystkie wiersze, które zawierają jakąś wartość NaN. Druga z nich zastępuje wartości NaN średnią.

Pierwszym z modeli jest Drzewo Decyzyjne, ma ono 2 wersje, 1 gdzie maksymalny poziom wynosi 9, a druga gdzie maksymalny poziom drzewa wynosi 5. Kod wygląda następująco:

In [139]:
from sklearn.tree import DecisionTreeClassifier

def classify(df, train_size):
    train_set, test_set = train_test_split(df, train_size=train_size)
    classifier = DecisionTreeClassifier(criterion="entropy", max_depth=9, min_samples_leaf=2, splitter="best",
                                        class_weight=None, random_state=101)
    classifier.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])
    accurarcy = classifier.score(test_set.iloc[:, :-1], test_set.iloc[:, -1])
    predicted = classifier.predict(test_set.iloc[:, :-1])
    confusionMatrix = confusion_matrix(test_set.iloc[:, -1], predicted)
    return accurarcy, confusionMatrix


def classifyMax(df, train_size, max_depth):
    train_set, test_set = train_test_split(df, train_size=train_size, random_state=2137)
    classifier = DecisionTreeClassifier(max_depth=max_depth)
    classifier.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])
    accurarcy = classifier.score(test_set.iloc[:, :-1], test_set.iloc[:, -1])
    predicted = classifier.predict(test_set.iloc[:, :-1])
    confusionMatrix = confusion_matrix(test_set.iloc[:, -1], predicted)
    return accurarcy, confusionMatrix

In [140]:
from Methods import DecisionTree
from Methods import Preprocessing

TRAIN_SIZE = 0.9


def main():
    dataD = Preprocessing.withDeletion(getData("./Data/winequality.csv"))
    dataM = Preprocessing.withMean(getData("./Data/winequality.csv"))

    getResultsOfDT(dataD, dataM)



def printResult(result):
    print("Accurarcy: ", result[0])
    # print("Confusion Matrix: ")
    # print(result[1])
    return ""


def getResultsOfDT(dataD, dataM):
    print("Without max-depth: ")
    print("Without normalization:")
    print("DT w/ Deletion: ")
    print(printResult(DecisionTree.classify(dataD, TRAIN_SIZE)), end='')
    print("DT w/ Mean: ")
    print(printResult(DecisionTree.classify(dataM, TRAIN_SIZE)), end='')
    print("With normalization:")
    print("DT w/ Deletion: ")
    print(printResult(DecisionTree.classify(Preprocessing.normalization(dataD), TRAIN_SIZE)), end='')
    print("DT w/ Mean: ")
    print(printResult(DecisionTree.classify(Preprocessing.normalization(dataM), TRAIN_SIZE)), end='')
    print("With max-depth: ")
    print("Without normalization:")
    print("DT w/ Deletion: ")
    print(printResult(DecisionTree.classifyMax(dataD, TRAIN_SIZE, 5)), end='')
    print("DT w/ Mean: ")
    print(printResult(DecisionTree.classifyMax(dataM, TRAIN_SIZE, 5)), end='')
    print("With normalization:")
    print("DT w/ Deletion: ")
    print(printResult(DecisionTree.classifyMax(Preprocessing.normalization(dataD), TRAIN_SIZE, 5)), end='')
    print("DT w/ Mean: ")
    print(printResult(DecisionTree.classifyMax(Preprocessing.normalization(dataM), TRAIN_SIZE, 5)), end='')

main()

Without max-depth: 
Without normalization:
DT w/ Deletion: 
Accurarcy:  0.714064914992272
DT w/ Mean: 
Accurarcy:  0.6984615384615385
With normalization:
DT w/ Deletion: 
Accurarcy:  0.7094281298299846
DT w/ Mean: 
Accurarcy:  0.7415384615384616
With max-depth: 
Without normalization:
DT w/ Deletion: 
Accurarcy:  0.6877897990726429
DT w/ Mean: 
Accurarcy:  0.6938461538461539
With normalization:
DT w/ Deletion: 
Accurarcy:  0.6877897990726429
DT w/ Mean: 
Accurarcy:  0.6938461538461539


Wyniki są następujące, dla drzewa w wersji z 9 poziomami wyniki są lepsze niż dla tego z 5 poziomami. Przy 5 poziomach normalizacja nie miała znaczenia, gdzie w przypadku 9 poziomów
normalizacja pogorszyła wyniki. Ustalenie 9 poziomów wynikało z przetestowania metod z wieloma roznymi parametrami przy użyciu GridSearchCV. W tym wypadku usunięcie danych było lepszym wyjściem.

Kolejnym modelem jest Naiwny Bayes, dla którego kod wygląda następująco:

In [141]:
from sklearn.naive_bayes import GaussianNB
from Methods import NaiveBayes_KN as NBK

def classifyNB(df, train_size):
    train_set, test_set = train_test_split(df, train_size=train_size, random_state=2137)

    classifier = GaussianNB()
    classifier.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])
    accurarcy = classifier.score(test_set.iloc[:, :-1], test_set.iloc[:, -1])
    predicted = classifier.predict(test_set.iloc[:, :-1])
    confusionMatrix = confusion_matrix(test_set.iloc[:, -1], predicted)
    return accurarcy, confusionMatrix

In [142]:
def main():
    dataD = Preprocessing.withDeletion(getData("./Data/winequality.csv"))
    dataM = Preprocessing.withMean(getData("./Data/winequality.csv"))

    getResultsOfNB(dataD, dataM)

def getResultsOfNB(dataD, dataM):
    print("Without normalization:")
    print("NB w/ Deletion: ")
    print(printResult(NBK.classifyNB(dataD, TRAIN_SIZE)), end='')
    print("NB w/ Mean: ")
    print(printResult(NBK.classifyNB(dataM, TRAIN_SIZE)), end='')
    print("With normalization:")
    print("NB w/ Deletion: ")
    print(printResult(NBK.classifyNB(Preprocessing.normalization(dataD), TRAIN_SIZE)), end='')
    print("NB w/ Mean: ")
    print(printResult(NBK.classifyNB(Preprocessing.normalization(dataM), TRAIN_SIZE)), end='')

main()

Without normalization:
NB w/ Deletion: 
Accurarcy:  0.6259659969088099
NB w/ Mean: 
Accurarcy:  0.6307692307692307
With normalization:
NB w/ Deletion: 
Accurarcy:  0.6136012364760433
NB w/ Mean: 
Accurarcy:  0.6230769230769231


Wyniki dla normalizacji jak i bez niej są bardzo podobne. Minimalnie na korzyść tych bez normalizacji. Lepsze wyniki wychodzą dla danych z uzupełnionymi brakami.

Kolejnym modelem jest kNN, dla którego kod wygląda następująco:

In [143]:
from sklearn.neighbors import KNeighborsClassifier

def classifyKN(df, train_size, neighbors):
    train_set, test_set = train_test_split(df, train_size=train_size, random_state=2137)

    classifier = KNeighborsClassifier(n_neighbors=neighbors)
    classifier.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])
    accurarcy = classifier.score(test_set.iloc[:, :-1], test_set.iloc[:, -1])
    predicted = classifier.predict(test_set.iloc[:, :-1])
    confusionMatrix = confusion_matrix(test_set.iloc[:, -1], predicted)
    return accurarcy, confusionMatrix

In [144]:

def main():
    dataD = Preprocessing.withDeletion(getData("./Data/winequality.csv"))
    dataM = Preprocessing.withMean(getData("./Data/winequality.csv"))
    getResultsOfKNN(dataD, dataM)

def getResultsOfKNN(dataD, dataM):
    print("Without normalization:")
    for i in range(10, 13):
        print("KNN w/ Deletion: ")
        print(printResult(NBK.classifyKN(dataD, TRAIN_SIZE, i)), end='')
        print("KNN w/ Mean: ")
        print(printResult(NBK.classifyKN(dataM, TRAIN_SIZE, i)), end='')
    print("With normalization:")
    for i in range(10, 13):
        print("KNN w/ Deletion: ")
        print(printResult(NBK.classifyKN(Preprocessing.normalization(dataD), TRAIN_SIZE, i)), end='')
        print("KNN w/ Mean: ")
        print(printResult(NBK.classifyKN(Preprocessing.normalization(dataM), TRAIN_SIZE, i)), end='')

main()

Without normalization:
KNN w/ Deletion: 
Accurarcy:  0.6367851622874807
KNN w/ Mean: 
Accurarcy:  0.6230769230769231
KNN w/ Deletion: 
Accurarcy:  0.6290571870170015
KNN w/ Mean: 
Accurarcy:  0.6215384615384615
KNN w/ Deletion: 
Accurarcy:  0.6259659969088099
KNN w/ Mean: 
Accurarcy:  0.6246153846153846
With normalization:
KNN w/ Deletion: 
Accurarcy:  0.7279752704791345
KNN w/ Mean: 
Accurarcy:  0.7184615384615385
KNN w/ Deletion: 
Accurarcy:  0.7279752704791345
KNN w/ Mean: 
Accurarcy:  0.7076923076923077
KNN w/ Deletion: 
Accurarcy:  0.7295208655332303
KNN w/ Mean: 
Accurarcy:  0.7107692307692308


Najlepsze wyniki są dla n=12, określiłem to przy użyciu tej pętli tutaj oraz również przy użyciu GridSearchCV. Przy użyciu normalizacji wyniki są zdecydowanie lepsze z nią. Wyniki są również
lepsze dla uzupełnienia braków średnią niż usunięciem danych.

Ostatnim modelem jest sieć neuronowa, dla której kod wygląda następująco:

In [145]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix



def classify(df, train_size):
    train_set, test_set = train_test_split(df, train_size=train_size, random_state=2137)

    # param_grid = {
    #     'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 50, 25), (10, 20, 30, 40), (100,50,100)],
    #     'activation': ['relu', 'tanh'],
    #     'solver': ['adam', 'sgd'],
    # }
    #
    # classifier = MLPClassifier()
    # grid_search = GridSearchCV(classifier, param_grid, cv=5)
    # grid_search.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])
    # print(grid_search.best_params_)
    # print(grid_search.best_score_)

    classifier = MLPClassifier(hidden_layer_sizes=(100, 50, 25), activation='tanh', solver='adam', max_iter=2000)
    classifier.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])

    predicted = classifier.predict(test_set.iloc[:, :-1])
    confusionMatrix = confusion_matrix(test_set.iloc[:, -1], predicted)
    accurarcy = accuracy_score(test_set.iloc[:, -1], predicted)
    return accurarcy, confusionMatrix

Użyty znowu został GridSearchCV, aby dobrać najlepsze parametry. Wyniki dla sieci neuronowej są bardzo dobre, a najlepsze są dla 100, 50, 25 neuronów w kolejnych warstwach.
Wyniki są następujące:

In [136]:
from Methods import Neural

def main():
    dataD = Preprocessing.withDeletion(getData("./Data/winequality.csv"))
    dataM = Preprocessing.withMean(getData("./Data/winequality.csv"))

    getResultsOfNeural(dataD, dataM)


LAYERS = (10, 7)


def getResultsOfNeural(dataD, dataM):
    print("Without normalization:")
    print("Neural w/ Deletion: ")
    print(printResult(Neural.classify(dataD, TRAIN_SIZE, LAYERS)), end='')
    print("Neural w/ Mean: ")
    print(printResult(Neural.classify(dataM, TRAIN_SIZE, LAYERS)), end='')
    print("With normalization:")
    print("Neural w/ Deletion: ")
    print(printResult(Neural.classify(Preprocessing.normalization(dataD), TRAIN_SIZE, LAYERS)), end='')
    print("Neural w/ Mean: ")
    print(printResult(Neural.classify(Preprocessing.normalization(dataM), TRAIN_SIZE, LAYERS)), end='')

main()

Accurarcy:  0.7418856259659969
Neural w/ Mean: 
Accurarcy:  0.703076923076923
With normalization:
Neural w/ Deletion: 
Accurarcy:  0.7217928902627512
Neural w/ Mean: 
Accurarcy:  0.7307692307692307
Without normalization:
Neural w/ Deletion: 
Accurarcy:  0.7279752704791345
Neural w/ Mean: 
Accurarcy:  0.7169230769230769
With normalization:
Neural w/ Deletion: 
Accurarcy:  0.7465224111282844
Neural w/ Mean: 


Wyniki są lepsze dla danych z normalizacją oraz dla uzupełnienia danych średnią.

Macierze błedów dla wszystkich modeli wyglądają następująco:

In [137]:

def main():
    dataD = Preprocessing.withDeletion(getData("./Data/winequality.csv"))
    dataM = Preprocessing.withMean(getData("./Data/winequality.csv"))

    getResultsOfDT(dataD, dataM)
    getResultsOfNB(dataD, dataM)
    getResultsOfKNN(dataD, dataM)
    getResultsOfNeural(dataD, dataM)

    return 0


def getData(directory):
    data = pd.read_csv(directory)
    data = shuffle(data)
    data['quality'] = data['quality'].replace([3, 4, 5], 'low')
    data['quality'] = data['quality'].replace([6, 7], 'medium')
    data['quality'] = data['quality'].replace([8, 9], 'high')
    return data


def printResult(result):
    print("Confusion Matrix: ")
    print(result[1])
    return ""

if __name__ == "__main__":
    main()


Without max-depth: 
Without normalization:
DT w/ Deletion: 
Confusion Matrix: 
[[  4   0  14]
 [  1 179  57]
 [ 11  89 292]]
DT w/ Mean: 
Confusion Matrix: 
[[  7   0  18]
 [  3 162  64]
 [ 10  91 295]]
With normalization:
DT w/ Deletion: 
Confusion Matrix: 
[[  4   1  15]
 [  1 177  76]
 [  2  90 281]]
DT w/ Mean: 
Confusion Matrix: 
[[  3   0   9]
 [  4 163  80]
 [  6  79 306]]
With max-depth: 
Without normalization:
DT w/ Deletion: 
Confusion Matrix: 
[[  0   0  22]
 [  0 137  99]
 [  0  77 312]]
DT w/ Mean: 
Confusion Matrix: 
[[  0   0  16]
 [  0 152  95]
 [  0  85 302]]
With normalization:
DT w/ Deletion: 
Confusion Matrix: 
[[  0   0  22]
 [  0 137  99]
 [  0  77 312]]
DT w/ Mean: 
Confusion Matrix: 
[[  0   0  16]
 [  0 152  95]
 [  0  85 302]]
Without normalization:
NB w/ Deletion: 
Confusion Matrix: 
[[  4   1  17]
 [  2 107 127]
 [ 32  87 270]]
NB w/ Mean: 
Confusion Matrix: 
[[  2   2  12]
 [  1 130 116]
 [ 39  98 250]]
With normalization:
NB w/ Deletion: 
Confusion Matrix:

Podsumowująć w większości przypadków normalizacja danych daje lepsze wyniki, a uzupełnienie braków danych średnią daje lepsze wyniki niż samo usunięcie danych. Najlepszym modelem okazała się sieć neuronowa oraz kNN, gdzie wyniki były na poziomie 75%. Najgorszym modelem okazał się naiwny klasyfikator bayesowski, gdzie wyniki były na poziomie 60%. Wszystkie modele testowane były na 90% danych treningowych oraz 10% danych testowych, bo dla takich wyniki były najlepsze.